In [1]:
import os 
import sys 
import json 
import logging 
from pathlib import Path 
from functools import cache
from itertools import product

# Required when developing in a jupyter-notebook environment 
cur_path = os.path.abspath("../..")
if cur_path not in sys.path: 
    sys.path.append(cur_path)

import numpy as np 
import pandas as pd 
import altair as alt 
from altair import datum
from dotenv import load_dotenv
from subgrounds.subgrounds import Subgrounds, Subgraph
from subgrounds.subgraph import SyntheticField
from subgrounds.pagination import ShallowStrategy

# Required when developing in a jupyter-notebook environment 
assert load_dotenv('../../../.env')

from utils_notebook.utils import ddf, remove_prefix, load_subgraph, remove_keys
from utils_notebook.vega import (
    condition_union, 
    output_chart, 
    apply_css, 
    stack_order_expr, 
    wide_to_longwide, 
    chart_stack_area_overlay_line_timeseries,
)
from utils_notebook.testing import validate_season_series
from utils_notebook.constants import ADDR_BEANSTALK
from utils_notebook.queries import QueryManager
from utils_notebook.css import css_tooltip_timeseries_multi_colored

In [2]:
sg, bs = load_subgraph()
q = QueryManager(sg, bs) 

In [3]:
@cache
def query_field_daily_snapshots(**kwargs): 
    return q.query_field_daily_snapshots(fields=['season', 'timestamp', 'totalHarvestablePods', 'totalHarvestedPods', 'totalPods']) 

In [4]:
df_field = query_field_daily_snapshots(cache=1).copy()
df_field = df_field.rename(columns={
    'totalHarvestablePods': 'harvestable_pods', 
    'totalHarvestedPods': 'harvested_pods', 
    'totalPods': 'unharvestable_pods', 
})
df_field['harvestable_pods'] = df_field.harvestable_pods - df_field.harvested_pods 
df_field['unharvestable_ratio'] = df_field.unharvestable_pods / (
    df_field.harvestable_pods + df_field.harvested_pods + df_field.unharvestable_pods
)
df_field.tail()

,season,harvestable_pods,harvested_pods,timestamp,unharvestable_pods,unharvestable_ratio
315,7545,23.012786,5.756021e+07,2022-10-06 23:00:11,7.712513e+08,0.930551
316,7569,158.473439,5.756021e+07,2022-10-07 23:00:11,7.713930e+08,0.930563
317,7593,172.447969,5.756021e+07,2022-10-08 23:00:11,7.714169e+08,0.930565
318,7617,172.447969,5.756021e+07,2022-10-09 23:00:11,7.714169e+08,0.930565
319,7635,172.447969,5.756021e+07,2022-10-10 17:00:11,7.714214e+08,0.930565


In [5]:
id_vars = ['timestamp', 'season']
value_vars = [
   'unharvestable_pods', 'harvested_pods', 'harvestable_pods', 'unharvestable_ratio'
]
df = wide_to_longwide(df_field, "timestamp", id_vars, value_vars, sort_col="timestamp") 
df.head()

,timestamp,season,variable,value,unharvestable_pods,harvested_pods,harvestable_pods,unharvestable_ratio
0,2021-08-07 00:06:08,25,unharvestable_pods,4899.059312,4899.059312,0.000000,5049.266341,0.492451
1,2021-08-07 00:06:08,25,harvested_pods,0.000000,4899.059312,0.000000,5049.266341,0.492451
2,2021-08-07 00:06:08,25,harvestable_pods,5049.266341,4899.059312,0.000000,5049.266341,0.492451
3,2021-08-07 00:06:08,25,unharvestable_ratio,0.492451,4899.059312,0.000000,5049.266341,0.492451
4,2021-08-08 00:08:15,49,unharvestable_pods,2502.841552,2502.841552,15526.430505,337.098080,0.136273


In [6]:
print(df.timestamp.dtype)

datetime64[ns]


In [11]:
colors = {
    'unharvestable_pods': '#ff595e', 
    'harvested_pods': "#8ac926", 
    'harvestable_pods': "#1982c4", 
    'unharvestable_ratio': "#6a4c93", 
} 
c = (
    chart_stack_area_overlay_line_timeseries(
        df, 
        "timestamp", 
        value_vars, 
        ['unharvestable_pods', 'harvested_pods', 'harvestable_pods',], 
        "Pod Line Breakdown", 
        tooltip_formats={
            'unharvestable_pods': ',d', 
            'harvested_pods': ',d', 
            'harvestable_pods': ',d', 
            'unharvestable_ratio': '.0%', 
        },
        yaxis_area_kwargs=dict(title="Pods", format=".3~s"), 
        yaxis_line_kwargs=dict(title="Ratio", format=".0%"), 
        separate_y_axes=True, 
        color_map=colors, 
        show_exploit_rule=True, 
        exploit_day=16, 
    )
)

css_lines = css_tooltip_timeseries_multi_colored(value_vars, colors) 
css = "\n".join(css_lines)

apply_css("")
# apply_css(css)

c

alt.LayerChart(...)

In [10]:
# TODO: update css for this chart 
output_chart(c, css=css)

<IPython.core.display.JSON object>